In [1]:
# 모듈 설정
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import pandas as pd
import os
import datetime
from urllib import parse

In [2]:
# 코드 엑셀파일 불러오기
code_df = pd.read_csv('code_a.csv', encoding='cp949')
code_df

,회사명,종목코드
0,GS글로벌,1250
1,HSD엔진,82740
2,KG케미칼,1390
3,LG이노텍,11070
4,OCI,10060
5,SK네트웍스,1740
6,SK이노베이션,96770
7,STX,11810
8,WISCOM,24070
9,갤럭시아에스엠,11420


In [3]:
# 저장파일 함수 지정
def crawling(new_code): #new_code = 종목코드
    df_crawling = pd.DataFrame()
    
    
    ###
    temp_page_url = 'https://finance.naver.com/item/news_news.nhn?code={}&page={}&sm=title_entity_id.basic&clusterId='.format(new_code, 1) 
    #request
    
    lastPageResp = requests.get(temp_page_url)
    lastPagehtml = lastPageResp.text
    
    #soup초기화
    soup = BeautifulSoup(lastPagehtml)
    
    href = soup.find('td', class_='pgRR').a['href']
    url = parse.urlparse(href)
    
    #Http URL Parameter를 다루려면 parse_qs() 메서드를 사용
    #parse_qs() 는 지정된 쿼리스트링을 해석하여 dict() 로 반환
    url_query = parse.parse_qs(url.query)
    
    pg = url_query['page'][0]
        
    ##400page가 넘는 경우 반복되는 news만 나오므로 400 = pg
    if int(pg) >= int(401):
        pg = int(400)
    else:
        pg = pg
    
    ###      

    for page in range(1,int(pg)+1):
        #종목에서 더 많은 페이지를 설정하는 경우, 마지막페이지만 계속 반복
        page_url = 'https://finance.naver.com/item/news_news.nhn?code={}&page={}&sm=title_entity_id.basic&clusterId='.format(new_code,page)
        resp = requests.get(page_url)
        html = resp.text
        soup = BeautifulSoup(html)
        
        a_tags = soup.select('a.tit')
        title_text=[]    
        link_text=[]
        for a_tag in a_tags:
            title_text.append(a_tag.text)
            link_text.append(a_tag['href'])

        td_tags = soup.select('td.date')   
        date_text=[]             
        for td_tag in td_tags:
            date_text.append(td_tag.string)  #.string하니까 시간만 정제됨!!!

        test_dict={}
        test_dict['date']=date_text
        test_dict['link']=link_text
        test_dict['title']=title_text  #title이라는 키값을 가지는 title_text 리스트의 요소들은  dict로 전환
    
        df_crawling = df_crawling.append(pd.DataFrame(test_dict), ignore_index=True)
    return df_crawling

def save_csv(df,new_code):  #def save_csv(crawling(code),code))
    
    #경로 찾기, 세이브 파일 보관 폴더 만들기
    BASE_DIR = os.getcwd()
    SAVE_DB_DIR = os.path.join(BASE_DIR, new_name)
    
    if not os.path.exists(SAVE_DB_DIR):
        os.makedirs(SAVE_DB_DIR)
    
    #텍스트 파일 만들기
    temp_name = f"{new_name}_{new_code}_{str(datetime.date.today())}.xlsx"
    txt_name = os.path.join(SAVE_DB_DIR, temp_name)
    
    df.to_excel(txt_name, sheet_name='sheet1')
    print("저장 완료. 프로그램을 종료합니다.")

In [4]:
def report(new_code):
    df_report = pd.DataFrame()
    period_year = 1      #오늘로부터 기간(year) 
    end_day = str(datetime.date.today())[:10]
    start_day = str(int(end_day[:4])-period_year)+end_day[4:]
    domain_url = 'https://www.wisereport.co.kr/wiseReport/reports/company.aspx'
    cookies = {
        'ASP.NET_SessionId': 'fct10vparoa4tz5t1smymwv0'
    }

    page_num=0
    report_list = []
    for i in range(1,10000):
        form_data = {
            'startDT': start_day,
            'endDt': end_day,
            'langTyp': 1,
            'searchTyp': None,
            'searchVal': None,
            'reVal': 'and a.cmp_cd = |{}|'.format(new_code),
            'perPage': 25,
            'curPage': i,
            'sortCol': 'anl_dt',
            'sortTyp': 'd',
            'flag': None,
            'flashYN': 1,
            'userId': 'gaps5_000543'
        }
        resp = requests.post(domain_url, cookies = cookies, data = form_data)
        soup = BeautifulSoup(resp.content,'html.parser')
        tg_contents = soup.find('tbody')

        if soup.find('tbody').find('tr').find('td').attrs['class'] == ['lst_center', 'end']:
            break
        else:
            page_num+=1
#             print('{}페이지를 가져오겠습니다.'.format(page_num))
    
        new_list=[]
        for content in tg_contents.contents[:-2]:
        
            new_dict = {}

            new_dict['title'] = content.find_all('td')[1].find('div').find('a').text

            new_dict['time']  = content.find('td').text

            summary_tag = content.find_all('td')[1].attrs['data-content']
            summary_soup = BeautifulSoup(summary_tag)
            summary_chunk = summary_soup.text
            summary_list = summary_chunk.split('▶')[1:]
            for i in range(len(summary_list)-1):
                summary_list[i] = summary_list[i][:-2]
            new_dict['summary'] = summary_list

            new_list.append(new_dict)

        report_list.extend(new_list)
    return report_list

In [5]:
def price(new_code):
    price_list = []
    url = 'https://finance.naver.com/item/sise_day.nhn'
    for num in range(1,579):
        params = {
            'code' : new_code,
            'page' : '{}'.format(num)
        }
        resp = requests.get(url, params = params)
        soup = BeautifulSoup(resp.content, 'html.parser')
        for tr in soup.find_all('tr')[2:14]:
            new_dict = {}
            td_list = tr.find_all('td')
            if not td_list[0].text.strip():
                continue
            tdtext_list = ['날짜', '종가', '전일비', '시가', '고가', '저가', '거래량']
            for i in range(7):
                new_dict[tdtext_list[i]] = td_list[i].text.strip()
            price_list.append(new_dict)
#     from pprint import pprint
#     pprint(result)
    return price_list

In [6]:
# 회사이름 쳐서 코드 불러오기
name = input('회사이름을 적어주세요 : ')
new_name = name.upper()
a = code_df[code_df['회사명'].isin([new_name])]
# name은 인풋 출력함수이므로 그 자체로 str, 별도의 ' 를 사용할 필요가 없다.
# print(a)
while True:
    if a.size == 0:
        print('정확한 회사명을 입력하세요')
        # 특정 문자열이 포함된 리스트 뽑기
        compony_ex = code_df[code_df.회사명.str.contains(new_name) == True]
        print(compony_ex)
        name = input('회사이름을 적어주세요 : ')
        new_name = name.upper()
        a = code_df[code_df['회사명'].isin([new_name])]
    else:
        code = a['종목코드'].values[0]
        new_code = '%06d' %code
#         print(new_code)
        break

df_news = crawling(new_code)

# 일일주가
price_list = []
url = 'https://finance.naver.com/item/sise_day.nhn'
for num in range(1,30):
    params = {
        'code' : new_code,
        'page' : '{}'.format(num)
    }
    resp = requests.get(url, params = params)
    soup = BeautifulSoup(resp.content, 'html.parser')
    for tr in soup.find_all('tr')[2:14]:
        new_dict = {}
        td_list = tr.find_all('td')
        if not td_list[0].text.strip():
            continue
        tdtext_list = ['날짜', '종가', '전일비', '시가', '고가', '저가', '거래량']
        for i in range(7):
            new_dict[tdtext_list[i]] = td_list[i].text.strip()
        price_list.append(new_dict)

        
# 리포트
period_year = 1      #오늘로부터 기간(year) 
end_day = str(datetime.date.today())[:10]
start_day = str(int(end_day[:4])-period_year)+end_day[4:]
domain_url = 'https://www.wisereport.co.kr/wiseReport/reports/company.aspx'
cookies = {
    'ASP.NET_SessionId': 'fct10vparoa4tz5t1smymwv0'
}

page_num=0
report_list = []
for i in range(1,10000):
    form_data = {
        'startDT': start_day,
        'endDt': end_day,
        'langTyp': 1,
        'searchTyp': None,
        'searchVal': None,
        'reVal': 'and a.cmp_cd = |{}|'.format(new_code),
        'perPage': 25,
        'curPage': i,
        'sortCol': 'anl_dt',
        'sortTyp': 'd',
        'flag': None,
        'flashYN': 1,
        'userId': 'gaps5_000543'
    }
    resp = requests.post(domain_url, cookies = cookies, data = form_data)
    soup = BeautifulSoup(resp.content,'html.parser')
    tg_contents = soup.find('tbody')

    if soup.find('tbody').find('tr').find('td').attrs['class'] == ['lst_center', 'end']:
        break
    else:
        page_num+=1
#             print('{}페이지를 가져오겠습니다.'.format(page_num))

    new_list=[]
    for content in tg_contents.contents[:-2]:

        new_dict = {}

        new_dict['title'] = content.find_all('td')[1].find('div').find('a').text

        new_dict['time']  = content.find('td').text

        summary_tag = content.find_all('td')[1].attrs['data-content']
        summary_soup = BeautifulSoup(summary_tag)
        summary_chunk = summary_soup.text
        summary_list = summary_chunk.split('▶')[1:]
        for i in range(len(summary_list)-1):
            summary_list[i] = summary_list[i][:-2]
        new_dict['summary'] = summary_list

        new_list.append(new_dict)
    report_list.extend(new_list)

    
    
while True:
    try:
        num=int(input("저장하시겠습니까?(1-저장, 2-종료) : "))
        print(num)
    
        if not num in [1,2]:
            print("잘못된 명령입니다. 다시 입력해주세요.")
            continue
        
        elif num == 1 :
            save_csv(df_news, new_code)

            
            BASE_DIR = os.getcwd()
            
            SAVE_DB_DIR = os.path.join(BASE_DIR, new_name)
            file_path1 = os.path.join(SAVE_DB_DIR, 'price.csv')
            file_path2 = os.path.join(SAVE_DB_DIR, 'report.csv')
            price = pd.DataFrame(price_list, columns = ['날짜', '종가', '전일비', '시가', '고가', '저가', '거래량'])
            price.to_csv(file_path1, encoding = 'CP949')
            
            report = pd.DataFrame(report_list, columns = ['time', 'title', 'summary'])
            report.to_csv(file_path2, encoding = 'CP949')
            
            
            break    
    
        else:
            print("프로그램을 종료합니다.")
            break
        
    except ValueError:
        print("잘못된 명령입니다. 다시 입력해주세요.")
        continue

회사이름을 적어주세요 : 삼성전자


AttributeError: 'NoneType' object has no attribute 'find'

# 실험정신

In [3]:
temp_page_url = 'https://finance.naver.com/item/news_news.nhn?code=005930&page=1&sm=title_entity_id.basic&clusterId=' 
    #request
    
lastPageResp = requests.get(temp_page_url)
lastPagehtml = lastPageResp.text
    
    #soup초기화
soup = BeautifulSoup(lastPagehtml)
    
href = soup.find('td', class_='pgRR').a['href']
url = parse.urlparse(href)

In [4]:
temp_page_url

'https://finance.naver.com/item/news_news.nhn?code=005930&page=1&sm=title_entity_id.basic&clusterId='

In [5]:
lastPageResp

<Response [200]>

In [6]:
lastPagehtml

'\n\n\n\n\n\n\n\n<html lang="ko">\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=euc-kr">\n<title>네이버 금융</title>\n<link rel="stylesheet" type="text/css" href="/css/newstock.css?20190529210744">\n<link rel="stylesheet" type="text/css" href="/css/common.css?20190529210744">\n<link rel="stylesheet" type="text/css" href="/css/layout.css?20190529210744">\n<link rel="stylesheet" type="text/css" href="/css/main.css?20190529210744">\n<link rel="stylesheet" type="text/css" href="/css/newstock2.css?20190529210744">\n<link rel="stylesheet" type="text/css" href="/css/newstock3.css?20190529210744">\n<link rel="stylesheet" type="text/css" href="/css/world.css?20190529210744">\n</head>\n<body>\n<script type="text/javascript" src="/js/jindo.min.ns.1.5.3.euckr.js?20190529210744"></script>\n<script language="JavaScript">\ndocument.domain="naver.com";\nvar nsc="finance.stockend";\n\nfunction mouseOver(obj){\n  obj.style.backgroundColor="#f6f4e5";\n}\nfunction mouseOut(obj){\n  obj.s

In [7]:
soup

<html lang="ko">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>네이버 금융</title>
<link href="/css/newstock.css?20190529210744" rel="stylesheet" type="text/css"/>
<link href="/css/common.css?20190529210744" rel="stylesheet" type="text/css"/>
<link href="/css/layout.css?20190529210744" rel="stylesheet" type="text/css"/>
<link href="/css/main.css?20190529210744" rel="stylesheet" type="text/css"/>
<link href="/css/newstock2.css?20190529210744" rel="stylesheet" type="text/css"/>
<link href="/css/newstock3.css?20190529210744" rel="stylesheet" type="text/css"/>
<link href="/css/world.css?20190529210744" rel="stylesheet" type="text/css"/>
</head>
<body>
<script src="/js/jindo.min.ns.1.5.3.euckr.js?20190529210744" type="text/javascript"></script>
<script language="JavaScript">
document.domain="naver.com";
var nsc="finance.stockend";

function mouseOver(obj){
  obj.style.backgroundColor="#f6f4e5";
}
function mouseOut(obj){
  obj.style.backgroundColor="#ffffff";


In [8]:
soup2 = BeautifulSoup(lastPageResp.content)
soup2

<html lang="ko">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>네이버 금융</title>
<link href="/css/newstock.css?20190529210744" rel="stylesheet" type="text/css"/>
<link href="/css/common.css?20190529210744" rel="stylesheet" type="text/css"/>
<link href="/css/layout.css?20190529210744" rel="stylesheet" type="text/css"/>
<link href="/css/main.css?20190529210744" rel="stylesheet" type="text/css"/>
<link href="/css/newstock2.css?20190529210744" rel="stylesheet" type="text/css"/>
<link href="/css/newstock3.css?20190529210744" rel="stylesheet" type="text/css"/>
<link href="/css/world.css?20190529210744" rel="stylesheet" type="text/css"/>
</head>
<body>
<script src="/js/jindo.min.ns.1.5.3.euckr.js?20190529210744" type="text/javascript"></script>
<script language="JavaScript">
document.domain="naver.com";
var nsc="finance.stockend";

function mouseOver(obj){
  obj.style.backgroundColor="#f6f4e5";
}
function mouseOut(obj){
  obj.style.backgroundColor="#ffffff";
